In [ ]:
import odc.geo.xr  # noqa: F401
from datacube import Datacube
from distributed import Client

from odc.algo import geomedian_with_mads
from odc.stac import configure_rio

In [ ]:
configure_rio(cloud_defaults=True, aws=dict(aws_unsigned=True))

In [ ]:
# Configure Dask
# dask.config.set(
#     {
#         "distributed.dashboard.link": "https://hub.csiro.easi-eo.solutions/{JUPYTERHUB_SERVICE_PREFIX}proxy/{port}/status"
#     }
# )

client = Client()
client

In [ ]:
# Study site in Tasmania
bbox = [146.2357, -43.6796, 147.147, -42.9305]

# All of tasmania
bbox = [144.0, -43.6, 148.0, -40.0]

lon = (bbox[0], bbox[2])
lat = (bbox[1], bbox[3])

year = "2017"
product = "s1_gamma0_geotif_scene"
resolution = 100

dc = Datacube()

In [ ]:
data = dc.load(
    product=product,
    lon=lon,
    lat=lat,
    time=year,
    dask_chunks=dict(x=2024, y=2024),
    output_crs="epsg:3577",
    resolution=(-resolution, resolution),
    group_by="solar_day",
)

data = data.where(data.vv > 0)

data

In [ ]:
geomad = geomedian_with_mads(data)

for band in ["vv", "vh"]:
    geomad["vv_mean"] = data[band].mean("time")

geomad

In [ ]:
computed = geomad.compute()

In [ ]:
mosaic_computed["mean_vv"].odc.explore(robust=True)

In [ ]:
mosaic_computed.mean_vv.odc.write_cog("s1_mosaic_2017_vv.tif")

In [ ]:
mosaic_computed.mean_vh.odc.write_cog("s1_mosaic_2017_vh.tif")